# Import the Libraries Needed

In [6]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

r  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

data = r.text

soup = BeautifulSoup(data, 'html.parser')

# Go Grab the table and pull the pieces out with Soup

Find the useful table
Work through each row
    - If the row has Not Assigned as a Borough - move on
    - If the row has no Neighborhood but a Borough - copy the Neighborhood
Save the dict into a dataframe

Now we have a dataframe but multiple rows per PostalCode and Borough

In [8]:
table = soup.find( "table", {"class":"wikitable sortable"} )

td = table.findAll('tr')[1:]

hoods = []

for data in td:
    col = data.find_all('td')
    details = {}
    for i,col in enumerate(col):
        if i == 1:
            details['Borough'] = (col.text.replace('\n',''))
            if details['Borough'] == 'Not assigned':
                break
        if i == 0:
            details['PostalCode'] = (col.text.replace('\n',''))
        if i == 2: 
            details['Neighborhood'] = (col.text.replace('\n',''))
            if details['Neighborhood'] == 'Not assigned':
                details['Neighborhood'] = details['Borough']
            hoods.append(details)

df=pd.DataFrame(hoods)
df=df[['PostalCode','Borough','Neighborhood']]
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


# Use Group by to do the work of concatinating the like Boroughs together

In [9]:
dfn=df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
dfn.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Displaying the Shape of the Final Dataset

In [10]:
dfn.shape

(103, 3)